# Full method for $d=3$ (Algorithm 2)

This notebook contains a complete working example of Algorithm 2 from the paper 

<i>Sampling triangulations of manifolds using Monte Carlo methods</i> by <a href="https://www.maths.usyd.edu.au/u/ega/">Eduardo Altmann</a> and <a href="https://sites.google.com/view/jonathan-spreer/">Jonathan Spreer</a>.


## Triangulations basics

A $d$-dimensional manifold $M$ is a topological space that locally looks like Euclidean $d$-space. A triangulation $T$ of $M$ is a subdivision of $M$ into $d$-simplices glued along their $(d-1)$-dimensional face such that the underlying space $|T|$ of $T$ (the $d$-simplices factored by their gluings) is homeomorphic to $M$.

The *face-vector* or *f-vector* $f(T) = (f_0, f_1, f_2, .... , f_d)$ of $T$ stores the number of $i$-dimensional simplices $f_i$ in $T$ in all dimensions $0 \leq i \leq d$. 

Example: The boundary of the tetrahedron $\Delta$ has face vector $f(\Delta) = (4,6,4)$.

## Regina basics

In this file, $T$ is always a triangulation of a $3$-dimensional manifold $M$, i.e., a set of tetrahedra, identified along their triangles. Data type is `regina.Triangulation3()`. Given a triagnulation `T`, type in `print T.detail()` to obtain an overview of what is stored in `T`, and print `T.fVector()` for its $f$-vector.

We can go from one triangulation `T` of $M$ to any other triangulation `T'` of $M$ by applying *Pachner moves* -- local modifications to `T` that change the isomorphism type of `T`, but not the topology of the underlying surface. Pachner moves in dimension three in regina are given by commands 

- `T.pachner(tet)`: Stellar subdivision of tetrahedron `tet` into four tetrahedra by placing a new vertex into the center of `tet`. This move adds one vertex, four edges, six triangles, and three tetrahedra to `T` (it replaces `tet` by four new tetrahedra). (This move is not used in Algorithm 2 below.)
- `T.pachner(t)`: Replaces two tetrahedra $\Delta_1$ and $\Delta_2$ joined along triangle `t` by three tetrahedra around a new edge with endpoints the two vertices of $\Delta_1$ and $\Delta_2$ opposite `t`. This move adds one edge, two triangles, and one tetrahedron.
- `T.pachner(e)`: This is the inverse of the previous move, taking three three tetrahedra joined around edge `e` and replacing it with two tetrahedra joined along the base triangle `t`. This move removes one edge, two triangles, and one tetrahedron.
- `T.pachner(v)`: Inverse of the stellar subdivision `T.pachner(tet)`. This move removes vertex `v` to merge four tetrahedra into one. (This move is not used in Algorithm 2 below.)

### Usage `c.pachner(face,check,perform)`

Rginas functions to perform moves are all organised the same way: a move always returns `True` or `False` on whether or not the operation was allowed / successful. It changes the underlying triangulation (if applicable), and takes as argument the face where the move is performed (the face that is going to be removed from the triangulation), and two booleans as optional arguments. The first one determining whether to check that the move is possible, the second one to determine whether to perform the move.

I.e., the command `T.pachner(v,True,False)`  takes as input a triangulation `T` and checks whether vertex `v` is contained in exactly four distinct tetrahedra, returns `True` or `False` accordingly, but does not alter `T`.

In [1]:
# Imports

# for timing
import time
# for exponential function
import math
# for random choices
import random
# import all functions etc. from regina
from regina import *

### Compute neighbouring triangulations

This function produces a dictionary of all `a`-neighbours (with isomorphism signatures as keys) of triangulation `iso` with f-vector `f`.

This function uses non-standard isomorphism signatures and hence requires `regina` version 7.3 or newer.

In [4]:
#######################################################
# finding all isomorphism types of neighbours
# in the Pachner graph
#
# iso:  isomorphism signature of current state 
#       (we must work with a canonical labelling 
#       for this to work)
# f:    f-vector of current state
# a:    1 = going up, 2 = going down
#######################################################
def neighbours(iso,f,a):
    nbrs = {}
    # going up (2-3-moves at all triangles contained in two distinct tetrahedra)
    if a == 1:
        for t in range(f[2]):
            # create copy of tri in standard iso sig labelling
            target = Triangulation3.fromIsoSig(iso)
            # test if move is possible and if so, perform it
            if target.pachner(target.triangle(t), True, False):
                target.pachner(target.triangle(t), False, True)
                # get isomorphism signature of result, add it to neighbours
                tiso = target.isoSig_RidgeDegrees()
                # add edge needed to flip to obtain this neighbour (in standard iso sig labelling)
                if not tiso in nbrs:
                    nbrs[tiso]=t
        return nbrs
    # going down (3-2-move at every edge of degree three in three distinct tetrahedra)
    if a == 2:
        for e in range(f[1]):
            # create copy of tri in standard iso sig labelling
            target = Triangulation3.fromIsoSig(iso)
            # test if move is possible and if so, perform it
            if target.pachner(target.edge(e), True, False):
                target.pachner(target.edge(e), False, True)
                # get isomorphism signature of result, add it to neighbours
                tiso = target.isoSig_RidgeDegrees()
                # add edge needed to flip to obtain this neighbour (in standard iso sig labelling)
                if not tiso in nbrs:
                    nbrs[tiso]=e
        return nbrs

### Choose move from proposal

This function takes a state triangulation given by isomorphism signature `iso` with f-vector `f` and paramter `gamma`. It computes a proposal, enumerates neighbours of `iso` and decides wether to perform the proposed move.

In [5]:
###############################################################################
# This chooses the next move (proposal and acceptance)
#
# iso:   isomorphism signature of current state 
#        (we must work with a canonical labelling 
#        for this to work)
# f:     f-vector of current state
# gamma: parameter for acceptance distribution
###############################################################################
def choosemove(iso, f, gamma):
    x = random.random()
    if x < math.exp((-1)*gamma*f[3]):
        a = 1
    else:
        a = 2
    ngbrs = neighbours(iso,f,a)
    num_ngbrs = len(ngbrs.keys())
    # random number for proposal to move or stay
    i = random.random()
    # setup done

    # go up (2-3)
    if a == 1:
        # stay where you are
        if i > float(num_ngbrs)/float(f[2]):
            return iso, f
        # move up (very likely)
        else:
            return random.choice(list(ngbrs.keys())), [f[0],f[1]+1,f[2]+2,f[3]+1]  
    # go down
    elif a == 2:
        # stay where you are
        if f[3] <= 2:
            return iso, f
        if i > float(num_ngbrs)/float(f[2]-2):
            return iso, f
        # go down (unlikely)
        else:
            return random.choice(list(ngbrs.keys())), [f[0],f[1]-1,f[2]-2,f[3]-1]
    return None

### Main function

This is the main function taking in see triangulation `iso` with f-vector `f`. It performs a random walk in the Pachner graph of length `steps` with parameter `gamma`. Parameter `verbose` decides print behaviour, `name` is the filename for the output file.

In [7]:
###############################################################################
# Iterate our method
#
# iso:      isomorphism signature of current state 
#           (we must work with a canonical labelling 
#           for this to work)
# f:        f-vector of current state
# steps:    number of steps
# gamma:    parameter for acceptance distribution
# interval: sample every "interval" steps
# offset:   discard first "offset" samples
# name:     filename
###############################################################################
def randomise(iso, f, steps, gamma, interval, offset, name):
    # initialise number of steps
    st = 0
    with open(name,"w") as fl:
        fl.write("")
    while st < steps + offset*interval-1:
        st += 1
        iso, f = choosemove(iso,f,gamma)
        if (interval != 0 and st % interval == 0 and st >= offset*interval):
            # open output file
            with open(name,"a") as fl:
                fl.write(iso+"\n")
            print("collecting triangulation",int((st-offset*interval)/interval+1),":",iso)
    return True

### Test the method

A short test of our method.

In [8]:
iso = "cMcabbgqs"
t = Triangulation3.fromIsoSig(iso)
# starting f-vector
f = t.fVector()
# number of steps (not including offset)
steps = 10000
# interval (number of steps between samples)
interval = 100
# offset (number of omitted samples at beginning of sequence)
offset = 100
# paramter
gamma = 1/10
# file to store raw sampling data in
filename = "out_S3_gamma_"+str(gamma)+".txt"


ttt = time.time()
trigs = randomise(iso, f, steps, gamma, interval, offset, filename)
print(time.time()-ttt)

collecting triangulation 1 : GLvAvLvLPzMvzPQQwAQPMQcadejiqqskmrAsyAxzzzuCAvDxxCDDFEFFbahugaaaboealmahfffnpboabftlfaguu
collecting triangulation 2 : FLvAvLvAQzwzLPwPALQQQkadejohnnpqktxpyswAtuEyyDzECDBDCEbahuagnfmabamognkqroaggxujrgbnrk
collecting triangulation 3 : DLvAvLAzzLMvQPwwAQQQcadeiljhokpqutoxtrwyuvACCAyABBCbahtaigibxiatctbbcawcahrcfbvvk
collecting triangulation 4 : CLwvvvvPLwMPMAPQMQQkacmisjnnqqqowpuvywtwxvyxABzABbgigmvwwcocsxfdiqjvnffbfsubgv
collecting triangulation 5 : BLwvvLzMwPwQzwQAPQQacihoijnlnmsttvwrsxxuzzyyyAAbgeiajsjnjgaitmorosbfpmmmmwc
collecting triangulation 6 : CLLvvwMzzzwAPLAQQMQkaejgnhklrqqxxzpttvuAxwvyyBBBAbpaohgkfxaaautfjvbrtrkonnaiqb
collecting triangulation 7 : DLvvLAvLAPMwLQzPQMMQcajkkipiprltnvuvrsztuwyAzyzABCCbalhvubgvrageeervlmbgbcojvfgjg
collecting triangulation 8 : DLvvAvAQPvMwzzAQAQAPcafejjhnilmrnsrvxyzztwwvAxyzCBCbccmioabeoagiqjpaaaoehcejkbsbb
collecting triangulation 9 : BLvAwLAvvzPQzAAQwQQadegkjhrssmuntqprwywvuzAzyAAbaxraaqatafgookvvliaafaaakqo
collec